In [1]:
from bs4 import BeautifulSoup
import requests
import random
import time
import json
from pymongo import MongoClient

In [2]:
header = {'user-agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36"}

In [4]:
url_header = 'https://www.booking.com/searchresults.html?label=gen173nr-1DCAEoggI46AdIM1gEaIkCiAEBmAExuAEHyAEM2AED6AEB-AECiAIBqAIDuAKJs6SRBsACAdICJGUyZTk4NGE2LWI2MjItNDQ0MS1hZTE5LThkNDE1Yjg5YjFmMtgCBOACAQ&sid=32b41a458c408ac53a6fda0dbb8fe5a4&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIM1gEaIkCiAEBmAExuAEHyAEM2AED6AEB-AECiAIBqAIDuAKJs6SRBsACAdICJGUyZTk4NGE2LWI2MjItNDQ0MS1hZTE5LThkNDE1Yjg5YjFmMtgCBOACAQ%3Bsid%3D32b41a458c408ac53a6fda0dbb8fe5a4%3Bsb_price_type%3Dtotal%3Bsrpvid%3D65fb95f2ef7c00af%26%3B&ss=California%2C+United+States&is_ski_area=0&ssne=Las+Vegas&ssne_untouched=Las+Vegas&checkin_year=2022&checkin_month=3&checkin_monthday=19&checkout_year=2022&checkout_month=3&checkout_monthday=27&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ss_raw=California&ac_position=0&ac_langcode=en&ac_click_type=b&dest_id=2279&dest_type=region&place_id_lat=35.8288&place_id_lon=-119.137276&search_pageview_id=c6af95f5bcde0076&search_selected=true'
url_list = []
for count in range(0, 40):
    offset = count * 25
    url = url_header + '&offset=' + str(offset)
    url_list.append(url)
# page = requests.get(url, headers = header)
# soup = BeautifulSoup(page.content, 'html.parser')

In [5]:
hotels = []
for url in url_list:
    page = requests.get(url, headers = header)
    soup = BeautifulSoup(page.content, 'html.parser')
    hotels.append(soup)
    time.sleep(random.randrange(5, 10))

In [6]:
def filter_number(string):
    result = ''
    for char in string:
        if char.isnumeric():
            result += char
    return result

In [7]:
client = MongoClient(port=27017)
database = client["admin"]
collection = database["booking_hotels"]

In [8]:
hotel_url_list = []
city_list = []
id_ = 1

for i in range(0, 40):
    result_page = hotels[i].findAll('div', {'data-testid':'property-card'})
    for each in result_page:
        # hotel name
        name = each.find('div',{'class':'fde444d7ef _c445487e2'}).text.strip()
        
        # hotel location: city
        city = each.find('span', {'class':'af1ddfc958 eba89149fb'}).text.strip()
        city_list.append(city)
        
        # hotel rating
        rating_temp = each.find('div', {'class': '_9c5f726ff bd528f9ea6'})
        if rating_temp is not None:
            rating = float(rating_temp.text.strip())
        else:
            rating = 'N/A'
        
        # room type
        room_type = each.find('span',{'class':'_c5d12bf22'}).text.strip()
        
        # provide free breakfast or not
        # 1: free breakfast included
        # 0: free breakfast not included
        break_temp = each.find('span', {'class': 'c79699a040'})
        if break_temp is not None:
            breakfast = 1
        else:
            breakfast = 0
        
        # price
        price_str = each.find('span', {'class':'fde444d7ef _e885fdc12'}).text.strip()
        total_price = filter_number(price_str)
        adr = int(total_price) / 8
        
        # reviews
        review_str = each.find('div', {'class':'_4abc4c3d5 _1e6021d2f _6e869d6e0'})
        if review_str is not None:
            review = filter_number(review_str.text.strip())
        else:
            review = 0
        
        # cancellation
        # 1: free cancellation
        # 0: no free cancellation
        cancel_temp = each.find('div', {'class': '_3abe99b47'})
        if cancel_temp is not None:
            cancel = 1
        else:
            cancel = 0
        
        # hotel detail pages url
        url = each.find('a', {'data-testid':'title-link'})['href']
        hotel_url_list.append(url)
        
        # star
        star = each.findAll('span', {'class': '_3ae5d40db _617879812 _6ab38b430'})
        star_rating = len(star)
        
        # preferred partner property: the property pays Booking.com a little bit more to get promoted
        # 1 is preferred partner property, 0 otherwise
        prefer_image = each.find('span', {'class': '_3ae5d40db a892149490 _5a3a8722d b557ea9051'})
        if prefer_image is not None:
            prefer = 1
        else:
            prefer = 0
        
        item = {'_id': id_, 'name': name, 'city': city, 'room_type': room_type, 'free_breakfast': breakfast,
                'free_cancellation': cancel, 'review': review, 'star_rating': star_rating, 'total_price': total_price, 
                'price_per_day': round(adr, 2), 'preferred_partner_property': prefer}
        collection.insert_one(item)
        id_+=1
        
#         print('name:',name, '\ncity:', city, '\nraitng:',rating, '\nbreakfast:', 
#               breakfast, '\nprice:', price, '\nroom:', room_type, '\nreviews:', review, '\ncancellation:', cancel, '\n')

In [9]:
address_book = []

for url in hotel_url_list:
    page = requests.get(url, headers = header)
    soup = BeautifulSoup(page.content, 'html.parser')
    address = soup.find('span', {'class':'hp_address_subtitle js-hp_address_subtitle jq_tooltip'}).text.strip()
    address_book.append(address)
    
    time.sleep(5)

In [10]:
i = 1

for address in address_book:
    api_geo_url = 'http://api.positionstack.com/v1/forward?access_key=12204865f85e092458ea60090afa1b78&query=' + address
    api_geo_page = requests.get(api_geo_url, headers=header)
    api_geo_doc = BeautifulSoup(api_geo_page.content, 'html.parser')
    geo_info = json.loads(str(api_geo_doc))
    
    lon = geo_info['data'][0]['longitude']
    lat = geo_info['data'][0]['latitude']
    
    time.sleep(5)
    
    api_pop_url = 'https://api.api-ninjas.com/v1/city?max_lat={}&max_lon={}'.format(lat, lon)
    api_pop_page = requests.get(api_pop_url, headers={'X-Api-Key': 'IXGvS0ImGQLtcUvoUcK+Yw==VzxABswHFD77vV4X'})
    api_pop_doc = BeautifulSoup(api_pop_page.content, 'html.parser')
    pop_info = json.loads(str(api_pop_doc))
    population = pop_info[0]['population']
    
    collection.update_one({'_id': i}, {'$set' : {'lon': lon, 'lat': lat, 'city_population': population}})
    i+=1
    
    time.sleep(5)